In [1]:
import os
import pandas as pd
import numpy as np
import bamboolib as bam
import wikipedia
ipynb_file_name = os.path.dirname(os.path.realpath('__file__')).rsplit("\\",1)[1]
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))
from bs4 import BeautifulSoup
import re
import requests
import json
from datetime import datetime as dt
from datetime import timedelta
import time

run_stamp = dt.now().strftime("%Y-%m-%d")

In [2]:
links = {'alabama':{ 
            'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Alabama'},
        'alaska': {
            'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_Alaska'},
         'arkansas': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Arkansas'},
         'california': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_California'},
         'colorado': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Colorado'},
         'connecticut': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Connecticut'},
         'delaware': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_Delaware'},
         'georgia': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Georgia'},
         'hawaii': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Hawaii'},
         'illinois': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Illinois'},
         'indiana': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Indiana'},
         'iowa': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Iowa'},
         'kansas': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Kansas'},
         'louisiana': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Louisiana'},
         'maine': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Maine'},
         'massachusetts': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Massachusetts'},
         'michigan': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Michigan'},
         'mississippi': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Mississippi'},
         'missouri': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Missouri'},
         'montana': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_Montana'},
         'nebraska': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Nebrasksa'},
         'nevada': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Nevada'},
         'new_jersey': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_New_Jersey'},
         'new_mexico': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_New_Mexico'},
         'new_york': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_New_York'},
         'north_carolina': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_North_Carolina'},
         'north_dakota': {
              'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_North_Dakota'},
         'ohio': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Ohio'},
         'oklahoma': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Oklahoma'},
         'oregon': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Oregon'},
         'pennsylvania': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Pennsylvania'},
         'rhode_island': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Rhode_Island'},
         'south_carolina': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_South_Carolina'},
         'south_dakota': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_South_Dakota'},
         'tennessee': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Tennessee'},
         'texas': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Texas'},
         'utah': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Utah'},
         'vermont': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_Vermont'},
         'virginia': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Virginia'},
         'washington': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Washington'},
         'west_virginia': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_West_Virginia'},
         'wyoming': {
             'site': 'https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_Wyoming'}
}

In [3]:
list_of_candidates = []

for link in links.items():
    url = (link[1]['site'])
    state = (str(link[0]))
    filename = os.path.join(link[0]) 

    def scrape(url, soup): 
        candidates = soup.find_all("td", {'class' : ["fn"]})
        
        for each_candidate in candidates: 
            candidate_name = each_candidate.select_one('b')
            list_of_candidates.append(candidate_name)

            list_of_candidates_without_none = list(filter(None, list_of_candidates))
            df = pd.DataFrame(list_of_candidates_without_none, dtype = 'object')
            df.columns = [str(column) for column in df.columns]
            df['CANDIDATE_NAME'] = df['0'].astype('string')
            df["CANDIDATE_NAME"] = df["CANDIDATE_NAME"].str.replace('<b>', '', regex=False)
            df["CANDIDATE_NAME"] = df["CANDIDATE_NAME"].str.replace('</b>', '', regex=False)
            df["CANDIDATE_NAME"] = df["CANDIDATE_NAME"].str.replace('<a href="/wiki/', '', regex=False)
            df["CANDIDATE_NAME"] = df["CANDIDATE_NAME"].str.replace('</a> (incumbent)', '', regex=False)
            df["CANDIDATE_NAME"] = df["CANDIDATE_NAME"].str.replace('>', '', regex=False)
            df["CANDIDATE_NAME"] = df["CANDIDATE_NAME"].str.replace('</a ', '', regex=False)
            df['CANDIDATE_NAME'] = [x.split('"')[-1] for x in df['CANDIDATE_NAME']]
            df = df.drop(columns=['0'])
            df_list = df.values.tolist()

            with open('general_election_candidates_state/' + filename + '_' + 'general_election_candidates_' + run_stamp + '.txt', 'w') as f: 
                for item in df_list: 
                    f.write("%s\n" % item)
        
    def browse_and_scrape(url): 
        try: 
            html_text = requests.get(url).text
            soup = BeautifulSoup(html_text, "html.parser")
            print(f"Currently scraping: {url}")
            
            if soup.find("li", class_ = "next") != None: 
                scrape(url, soup)
                time.sleep(3)
                page_number += 1
                browse_and_scrape(url, page_number)
            else: 
                scrape(url, soup)
                return True
            return True
        except Exception as e:
            return e

    if __name__ == "__main__": 

        print("Web scraping of {} has begun!".format(state))
        result = browse_and_scrape(url)
        if result == True: 
            print("Web scraping {} is complete!".format(state))
        else: 
            print(f"Something is wrong: {result}")

Web scraping of alabama has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Alabama
Something is wrong: '0'
Web scraping of alaska has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_Alaska
Web scraping alaska is complete!
Web scraping of arkansas has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Arkansas
Web scraping arkansas is complete!
Web scraping of california has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_California
Web scraping california is complete!
Web scraping of colorado has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Colorado
Something is wrong: '0'
Web scraping of connecticut has begun!
Currently scraping: https://en.wikipedia.org/wiki/2

C:\Users\Kohayward\Anaconda3\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



Web scraping texas is complete!
Web scraping of utah has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Utah
Web scraping utah is complete!
Web scraping of vermont has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_election_in_Vermont
Web scraping vermont is complete!
Web scraping of virginia has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Virginia
Web scraping virginia is complete!
Web scraping of washington has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_Washington
Web scraping washington is complete!
Web scraping of west_virginia has begun!
Currently scraping: https://en.wikipedia.org/wiki/2022_United_States_House_of_Representatives_elections_in_West_Virginia
Web scraping west_virginia is complete!
Web scraping of wyoming has 

In [4]:
from twilio.rest import Client
account_sid = os.environ.get('TWILIO_ACCOUNT_SID')
auth_token = os.environ.get('TWILIO_AUTH_TOKEN')
client = Client(account_sid, auth_token)
client.messages.create(from_ = os.environ.get('TWILIO_PHONE_NUMBER'),
to = os.environ.get('KH_PHONE_NUMBER'),
body = 'Your script -- {} -- has started running!'.format(ipynb_file_name))

<Twilio.Api.V2010.MessageInstance account_sid=ACb17e5173f634afd26596ed51d595bfe5 sid=SMd3eae9d6c2804989a954309d22701b52>

In [5]:
print(account_sid)

ACb17e5173f634afd26596ed51d595bfe5


In [6]:
account_sid = os.environ.get('TWILIO_ACCOUNT_SID')
auth_token = os.environ.get('TWILIO_AUTH_TOKEN')
print(account_sid)

ACb17e5173f634afd26596ed51d595bfe5
